### Парсинг файлов после MAST 

In [1]:
from bs4 import BeautifulSoup

In [3]:
with open("mast_interpro_1_cl.xml","r") as f:
    contents = f.read()
soup = BeautifulSoup(contents,'xml')

In [4]:
motifs = soup.motifs.findAll('motif')
length = [motif['length'] for motif in motifs]
length

['35', '31', '29']

На 24 месте в первом мотиве не должен стоять Cys. Проверим это и запишем проверенные сиквенсы в отдельные файл.

In [5]:
sequences = soup.sequences.findAll('sequence')

In [6]:
import re
reg = re.compile('[^a-zA-Z ]')

In [ ]:
Cys_seq_names = set()
Cys_seq = set()
for seq in sequences:
    for seg in seq.find_all('seg'):
        if bool(seg.find(idx = '0')) == True:
            if reg.sub('', seg.data.string)[int(seg.find(idx = '0')['pos']) - int(seg['start']) + 23] != 'C':
                Cys_seq_names.add(seq['name'])
                Cys_seq.add(seq)

In [ ]:
with open('Cysless_sequences_4.xml', 'w') as f:
    for i in Cys_seq:
        f.write(str(i) + '\n')

Отдельно запишем данные в формате fasta

In [ ]:
with open('protein_4.txt', 'r') as f:
    fasta = {}
    for line in map(lambda st: st.rstrip('\n'), f):
        if line.startswith('>'):
            name = line
        else:
            fasta.setdefault(name, []).append(line)

In [ ]:
new_file = []
for name in Cys_seq_names:
    new_file.append('>' + name + '\n')
    for s in fasta.get('>' + name):
        new_file.append(s + '\n')

In [ ]:
with open('Cysless_proteins_4.fasta', 'w') as f:
    f.write(''.join(new_file))import pandas as pd

### Парсинг результатов pfam

In [4]:
import pandas as pd

Переведем txt таблицу в удобный нам пандас.

In [62]:
columns = []
d = []
with open('pfam_4_cl.txt', 'r') as f:
    for line in f:
        if line.startswith('<'):
            columns = line.split('> <')
        else:
            d.append(line.split())
columns[0] = columns[0].strip('<')
columns[-1] = columns[-1].strip('>\n')

In [63]:
data = pd.DataFrame(d, columns=columns)
data.head()

,seq id,alignment start,alignment end,envelope start,envelope end,hmm acc,hmm name,type,hmm start,hmm end,hmm length,bit score,E-value,significance,clan
0,M1FCN8,32,128,22,129,PF13426.7,PAS_9,Domain,11,103,104,76.6,1.5e-21,1,CL0183
1,M1FCN8,148,251,148,251,PF13426.7,PAS_9,Domain,1,104,104,64.4,9.1e-18,1,CL0183
2,M1FCN8,265,416,263,416,PF00990.21,GGDEF,Domain,3,161,161,125.5,1.5e-36,1,CL0276
3,V4Y023,135,236,134,236,PF13426.7,PAS_9,Domain,2,104,104,42.9,4.7e-11,1,CL0183
4,V4Y023,257,361,255,361,PF13426.7,PAS_9,Domain,3,104,104,76.0,2.3e-21,1,CL0183


### Идентифекация LOV 

Отбор по отсутствию Cys уже прошел, поэтому необходимо отсеять по p-value. 

In [116]:
with open("Cysless_sequences_4.xml","r") as f:
    contents = f.read()
soup = BeautifulSoup(contents,'xml')

Отобрали dictionary нужных наименований со списком позиций трех мотивов (этих троек мб несколько в последовательности)

In [117]:
sequences = soup.sequences.find_all('sequence')
seqs = {}
for seq in sequences:
    for seg in seq.find_all('seg'):
        if (bool(seg.find(idx = '0')) == True and bool(seg.find(idx = '1')) == True and bool(seg.find(idx = '2')) == True):
            hits = seg.find_all('hit')
            if (float(hits[0]['pvalue']) <= 1e-15 and float(hits[1]['pvalue']) <= 1e-15 and float(hits[2]['pvalue']) <= 1e-15):
                seqs.setdefault(seq['name'], []).append([int(hits[0]['pos']), int(hits[1]['pos']), 
                                                         int(hits[2]['pos'])])

In [118]:
new_names = []
wrong = []
for k in seqs.keys():
    seq_data = data.loc[data['seq id'] == k] # data - таблица с результатами pfam, здесь отсев по id
    df = seq_data.loc[seq_data['clan'] == 'CL0183'] # здесь отсев по id клана (группы) домена PAS
    b = False
    for hit in seqs[k]:
        for i in range(df.shape[0]):
            if int(df.iloc[i, 3]) - hit[0] <= 2 and hit[2] + 28 - int(df.iloc[i, 4]) <= 10:
                b = True
    if not b:
        data.drop(data[data['seq id'] == k].index, inplace=True)
        wrong.append(k)
    else:
        new_names.append(k)   

In [119]:
with open('/home/dashik/projects/181015_conf_mipt_18/protein_4.txt', 'r') as f:
    fasta = {}
    for line in map(lambda st: st.rstrip('\n'), f):
        if line.startswith('>'):
            name = line
        else:
            fasta.setdefault(name, []).append(line)

In [120]:
new_file = []
for name in new_names:
    new_file.append('>' + name + '\n')
    for s in fasta.get('>' + name):
        new_file.append(s + '\n')

In [121]:
with open('final_proteins_4_cl.fasta', 'w') as f:
    f.write(''.join(new_file))

Проверим на пересечения

In [122]:
with open('/home/dashik/projects/181015_conf_mipt_18/With_Cys/final_proteins_4.fasta', 'r') as f:
    fasta = {}
    for line in map(lambda st: st.rstrip('\n'), f):
        if line.startswith('>'):
            name = line
        else:
            fasta.setdefault(name, []).append(line)

In [123]:
with open('final_proteins_4_cl.fasta', 'r') as f:
    fasta_cl = {}
    for line in map(lambda st: st.rstrip('\n'), f):
        if line.startswith('>'):
            name = line
        else:
            fasta_cl.setdefault(name, []).append(line)

In [124]:
repeat = list(fasta.keys() & fasta_cl.keys())
len(repeat)

77

In [125]:
for r in repeat:
    fasta_cl.pop(r)

In [126]:
new_file = []
for name in fasta_cl.keys():
    new_file.append(name + '\n')
    for s in fasta_cl.get(name):
        new_file.append(s + '\n')
with open('final_proteins_4_cl.fasta', 'w') as f:
    f.write(''.join(new_file))

Удалили повторы и объеденили финальные последоватеьнсти в оди файл, для построения дерева и выравнивания белковых последовательностей.

In [1]:
 # data - df pfam
'''Доступ к Uniprot'''
from Bio import ExPASy
from Bio import SwissProt
from tqdm import trange

In [2]:
def kingdom(s):
    handle = ExPASy.get_sprot_raw(s[0])
    try:
        record = SwissProt.read(handle)
        return record.organism_classification[0] + ' ' + record.organism_classification[1]
    except ValueError:
        return None

In [64]:
data['kingdom'] = None

In [65]:
with open('final_proteins_4_cl.fasta', 'r') as f:
    fasta = {}
    for line in map(lambda st: st.rstrip('\n'), f):
        if line.startswith('>'):
            name = line.lstrip('>')
        else:
            fasta.setdefault(name, []).append(line)        

In [66]:
len(fasta)

69

In [67]:
new_data = pd.DataFrame()
for k in fasta.keys():
    new_data = pd.concat([new_data, data.loc[data['seq id'] == k, :]], ignore_index=True)

In [69]:
for i in trange(400):
    new_data.iloc[i, 15] = kingdom(new_data.iloc[i,:])

100%|██████████| 400/400 [05:46<00:00,  1.12it/s]


In [68]:
new_data.shape

(400, 16)

In [70]:
tfile = open('domains_4_cl.csv', 'w')
tfile.write(new_data.to_csv())
tfile.close()

In [61]:
data.head()

,seq id,alignment start,alignment end,envelope start,envelope end,hmm acc,hmm name,type,hmm start,hmm end,hmm length,bit score,E-value,significance,clan,kingdom
0,A0A2T6DB03,7,113,7,117,PF00072.24,Response_reg,Domain,1,108,112,56.3,3.1e-15,1,CL0304,None
1,A0A2T6DB03,165,249,164,249,PF08447.12,PAS_3,Domain,2,89,89,58.6,5.5e-16,1,CL0183,None
2,A0A2T6DB03,275,381,273,381,PF13426.7,PAS_9,Domain,3,104,104,67.8,8e-19,1,CL0183,None
3,A0A2T6DB03,526,631,524,632,PF08448.10,PAS_4,Domain,3,109,110,34.7,1.6e-08,1,CL0183,None
4,A0A2T6DB03,647,706,645,710,PF00512.25,HisKA,Domain,3,63,67,29.4,5.8e-07,1,CL0025,None


#### Находим LOV

In [187]:
from bs4 import BeautifulSoup
with open("Cysless_sequences_4.xml","r") as f:
    contents = f.read()
soup = BeautifulSoup(contents,'xml')

sequences = soup.sequences.findAll('sequence')

In [188]:
seqs = {}
for seq in sequences:
    for seg in seq.find_all('seg'):
        if (bool(seg.find(idx = '0')) == True and bool(seg.find(idx = '1')) == True and bool(seg.find(idx = '2')) == True):
            hits = seg.find_all('hit')
            if (float(hits[0]['pvalue']) <= 1e-15 and float(hits[1]['pvalue']) <= 1e-15 and float(hits[2]['pvalue']) <= 1e-15):
                seqs.setdefault(seq['name'], []).append([int(hits[0]['pos']), int(hits[1]['pos']), 
                                                         int(hits[2]['pos'])])

In [189]:
with open('final_proteins_4_cl.fasta', 'r') as f:
    fasta = {}
    for line in map(lambda st: st.rstrip('\n'), f):
        if line.startswith('>'):
            name = line.lstrip('>')
        else:
            fasta.setdefault(name, []).append(line)

In [190]:
new_seq = {}
for key in seqs.keys():
    if key in fasta:
        new_seq[key] = seqs[key]
seqs = new_seq

In [191]:
data = pd.read_csv('domains_4_cl.csv')

In [172]:
def IfLOV(row):
    if row['clan'] == 'CL0183':
        for hit in seqs[row['seq id']]:
            if (int(row[3]) - hit[0]) <=5 and (hit[2] + 29 - int(row[4])) <= 15:
                return 'LOV'
            else: 
                return row['hmm name']
    else:
        return row['hmm name']

In [193]:
data.head()

,seq id,alignment start,alignment end,envelope start,envelope end,hmm acc,hmm name,type,hmm start,hmm end,hmm length,bit score,E-value,significance,clan,kingdom
0,M0D0B1,18,105,14,116,PF00072.24,Response_reg,Domain,14,102,112,44.9,1.100000e-11,1,CL0304,Archaea Euryarchaeota
1,M0D0B1,152,256,150,256,PF13426.7,LOV,Domain,3,104,104,72.9,2.200000e-20,1,CL0183,Archaea Euryarchaeota
2,M0D0B1,307,435,299,436,PF13185.6,GAF_2,Domain,9,137,138,34.6,2.100000e-08,1,CL0161,Archaea Euryarchaeota
3,M0D0B1,447,600,447,600,PF15915.5,BAT,Domain,1,156,156,122.6,1.300000e-35,1,No_clan,Archaea Euryarchaeota
4,M0D0B1,606,658,606,658,PF04967.12,HTH_10,Domain,1,53,53,73.5,8.300000e-21,1,CL0123,Archaea Euryarchaeota


In [192]:
data['hmm name'] = data.apply(IfLOV, axis = 1)

#### Находим TLOV 

In [194]:
from tqdm import tqdm

for name in tqdm(seqs.keys()):
    indexes = data[(data['seq id'] == name) & (data['hmm name'] == 'LOV')].index.values
    for i in indexes[:-1]:
        if (data.loc[i+1, 'hmm name'] == 'LOV' and 
            int(data.loc[i+1, 'envelope start']) - int(data.loc[i, 'envelope end']) <= 125):
            data.loc[i, 'envelope end'] = data.loc[i+1, 'envelope end']
            data.loc[i, 'hmm name'], data.loc[i+1, 'hmm name'] = 'TLOV', '_LOV'

100%|██████████| 69/69 [00:00<00:00, 363.84it/s]


In [84]:
data = data.loc[data['hmm name'] != '_LOV']

In [195]:
tfile = open('domains_LOV_4_cl.csv', 'w')
tfile.write(data.to_csv())
tfile.close()